In [1]:
import tensorflow as tf


In [3]:

import pandas as pd
import string
import re
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from matplotlib.ticker import PercentFormatter
import nltk
import spacy
import sys
from spacy.lang.en import English
import en_core_web_sm
from nltk.corpus import wordnet as wn
from nltk.stem.wordnet import WordNetLemmatizer

print(tf.__version__)  # 2.0.0-beta0

2.10.1


In [5]:
df = pd.read_csv("C:/bootcamp/tensorflow_gpu/data/chatgpt_short_clean_combined_en.csv", 
                 parse_dates=["at", "repliedAt"])

In [4]:
# Загрузка модели для обработки естественного языка (NLP) с использованием en_core_web_sm
nlp = en_core_web_sm.load()

# Инициализация парсера для английского языка
parser = English()

# Загрузка списка стоп-слов на английском языке из библиотеки nltk
en_stop = set(nltk.corpus.stopwords.words('english'))

# Функция для токенизации заголовка в список отдельных слов
def tokenize(text):
    """Эта функция токенизирует заголовок, создавая список отдельных слов."""
    lda_tokens = []
    tokens = parser(text)  # использование парсера, чтобы Python рассматривал список как слова
    for token in tokens:
        if token.orth_.isspace():  # игнорирование пробелов в заголовке, чтобы список токенов не содержал пробелы
            continue
        elif token.like_url:
            lda_tokens.append('URL')
        elif token.orth_.startswith('@'):
            lda_tokens.append('SCREEN_NAME')
        else:
            lda_tokens.append(token.lower_)   # токены (заголовки) уже в нижнем регистре
    return lda_tokens

# Функция для получения леммы (основной формы) слова
def get_lemma(word):
    """Эта функция лемматизирует слова в заголовке, приводя их к основной форме."""
    lemma = wn.morphy(word)  # конвертация слова в основную форму с использованием WordNet
    if lemma is None:
        return word
    else:
        return lemma
    
# Функция для подготовки текста к LDA
def prepare_text_for_lda(text):
    # Разбиение и токенизация заголовка в список слов
    tokens = tokenize(text)
    
    # Удаление заголовков с длиной 4 слова или меньше
    tokens = [token for token in tokens if len(token) > 4]
    
    # Удаление стоп-слов из заголовка
    tokens = [token for token in tokens if token not in en_stop]
    
    # Лемматизация слов в заголовке
    tokens = [get_lemma(token) for token in tokens]
    
    return tokens



In [7]:
# Создание пустого списка для хранения обработанных данных
text_data = []

# Проход по каждому заголовку в столбце 'headline' датафрейма df
for headline in df.content:
    # Подготовка текста для LDA с использованием определенной выше функции prepare_text_for_lda
    tokens = prepare_text_for_lda(headline)
    
    # Добавление обработанных токенов в список text_data
    text_data.append(tokens)

In [10]:
from gensim import corpora
import pickle

# Создание словаря (dictionary) для преобразования текстовых данных в мешок слов (bag-of-words)
dictionary = corpora.Dictionary(text_data)

# Преобразование каждого заголовка (списка слов) в формат мешка слов (bag-of-words). 
# В результате получаем список, где каждый элемент - это пара (ID слова, количество слова в тексте).
corpus = [dictionary.doc2bow(text) for text in text_data]

# Сохранение корпуса в файл с использованием библиотеки pickle
pickle.dump(corpus, open('corpus.pkl', 'wb'))

# Сохранение словаря в файл формата Gensim
dictionary.save('dictionary.gensim')
# Процесс создания словаря и корпуса занимает некоторое время


In [11]:
loaded_dictionary = corpora.Dictionary.load('dictionary.gensim')

# Print information about the loaded dictionary
print("Loaded Dictionary:")
print("Number of unique tokens:", len(loaded_dictionary))
print("Token frequencies:")
for token, idx in loaded_dictionary.token2id.items():
    print(f"Token: {token}, Index: {idx}, Frequency: {loaded_dictionary.dfs[idx]}")

Loaded Dictionary:
Number of unique tokens: 10719
Token frequencies:
Token: protective, Index: 0, Frequency: 3
Token: background, Index: 1, Frequency: 49
Token: black, Index: 2, Frequency: 46
Token: browser, Index: 3, Frequency: 466
Token: complete, Index: 4, Frequency: 92
Token: contrast, Index: 5, Frequency: 14
Token: gray, Index: 6, Frequency: 1
Token: letters, Index: 7, Frequency: 52
Token: question, Index: 8, Frequency: 1486
Token: arbina, Index: 9, Frequency: 1
Token: everything, Index: 10, Frequency: 669
Token: program, Index: 11, Frequency: 166
Token: software, Index: 12, Frequency: 152
Token: words, Index: 13, Frequency: 292
Token: amaze, Index: 14, Frequency: 1828
Token: helpful, Index: 15, Frequency: 1951
Token: intelligent, Index: 16, Frequency: 197
Token: loyal, Index: 17, Frequency: 6
Token: instal, Index: 18, Frequency: 139
Token: super, Index: 19, Frequency: 308
Token: suuuuuper, Index: 20, Frequency: 1
Token: analysis, Index: 21, Frequency: 40
Token: chatgpt, Index: 22

In [17]:
# Определение количества тем для модели LDA
NUM_TOPICS = [50]

# Создание модели LDA
# corpus: Список, представляющий корпус в формате bag-of-words (мешок слов)
# num_topics: Количество тем, которое мы хотим выделить
# id2word: Словарь, который преобразует идентификаторы слов обратно в слова
# passes: Количество проходов по корпусу в процессе обучения
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=50, id2word=dictionary, passes=300)

# Сохранение модели LDA в файл 'model5.gensim'
ldamodel.save('model5.gensim')

# Вывод тем, выделенных в процессе обучения
topics = ldamodel.print_topics(num_words=5)

# Вывод тем в виде списка тем, каждая из которых представляет собой набор слов
topics


[(9,
  '0.343*"helpful" + 0.255*"useful" + 0.105*"download" + 0.083*"recommend" + 0.033*"highly"'),
 (34,
  '0.141*"improve" + 0.128*"super" + 0.108*"nothing" + 0.056*"latest" + 0.050*"know"'),
 (1,
  '0.151*"feature" + 0.083*"website" + 0.075*"history" + 0.075*"voice" + 0.071*"access"'),
 (22,
  '0.251*"thanks" + 0.075*"happy" + 0.059*"human" + 0.056*"study" + 0.048*"openai"'),
 (11,
  '0.170*"available" + 0.097*"person" + 0.097*"waiting" + 0.062*"talking" + 0.050*"premium"'),
 (26,
  '0.182*"write" + 0.102*"writing" + 0.079*"fantastic" + 0.062*"story" + 0.056*"especially"'),
 (38,
  '0.184*"excellent" + 0.089*"tool" + 0.062*"important" + 0.049*"business" + 0.047*"shortcut"'),
 (33,
  '0.378*"better" + 0.172*"change" + 0.050*"given" + 0.044*"convenient" + 0.042*"satisfy"'),
 (36,
  '0.137*"without" + 0.105*"assistant" + 0.093*"love" + 0.089*"different" + 0.088*"personal"'),
 (37,
  '0.302*"would" + 0.182*"could" + 0.076*"language" + 0.049*"instead" + 0.029*"add"'),
 (6,
  '0.334*"than

In [18]:
dictionary = gensim.corpora.Dictionary.load('dictionary.gensim')
corpus = pickle.load(open('corpus.pkl', 'rb'))
lda = gensim.models.ldamodel.LdaModel.load('model5.gensim')
import pyLDAvis.gensim
lda_display = pyLDAvis.gensim.prepare(lda, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)